In [1]:
def summary(user_text):
    yt = YouTube(user_text)
    caption = yt.captions.get_by_language_code('ko.7XP2tGORuV4')
    if caption == None:
        caption = yt.captions.all()[0]
    text = caption.xml_captions
    불용어 = ['<[^가-힣]+>', '&[^가-힣]+;', '(박수 소리)', '\n', '( )', ':' , '(앵커)', '(기자)']
    for i in 불용어:
        text = re.sub(i,' ', text)
        
    kkma = Kkma()

    # text입력받아 kkma.sentences()를 이용해 문장단위로 나는 뒤 sentences로 리턴
    def text2sentences(text):
        sentences= kkma.sentences(text)  # text일 때 문장별로 리스트 만듦
        for idx in range(0, len(sentences)): #길이에 따라 문장 합침
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    sentences = text2sentences(text)
    twitter = Twitter()

    #불용어제거
    stopwords = ['( )','안녕하세요', '아', '으', '네' ,'코']
    def get_nouns(sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                    if noun not in stopwords and len(noun) > 1]))
        return nouns
    nouns = get_nouns(sentences)
    tfidf = TfidfVectorizer()
    cnt_vec = CountVectorizer()
    graph_sentence = []
    def build_sent_graph(sentence):
        tfidf_mat = tfidf.fit_transform(sentence).toarray()
        graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return graph_sentence

    sent_graph = build_sent_graph(nouns)
    def build_words_graph(sentence):
        cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
    words_graph, idx2word = build_words_graph(nouns)
    def get_ranks(graph, d=0.85):
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0
            link_sum = np.sum(A[:,id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B)
        return {idx: r[0] for idx, r in enumerate(ranks)}
    sent_rank_idx = get_ranks(sent_graph)
    sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)
    word_rank_idx = get_ranks(words_graph)
    sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)
    
    summarize = []
    index = []
    for idx in sorted_sent_rank_idx[:5]:
        index.append(idx)
        
    index.append(idx)
    
    for idx in index:
        summarize.append(sentences[idx])
        
    for text in summarize[:]:
        llist.append(text)
    
    m = list(set(llist))
    
    keywords = []
    index = []
    for idx in sorted_word_rank_idx[:15]:
        index.append(idx)
        
    for idx in index:
        keywords.append(idx2word[idx])
        
    keywords = [each_word for each_word in keywords
                if each_word not in stop_words]
        
    keyword = keywords[:5]
    return m, keyword

<>:29: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:29: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/ls/xl42zdxs2992bdl9ypqdvtn80000gn/T/ipykernel_59762/3153922296.py:29: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [1]:
with open('C:/Users/82102/Desktop/4월 프로젝트/total_texts.txt', 'r', encoding='UTF8') as file: 
    a = file.readlines()
a

['한국은행은 오늘 금융통화위원회를 열고 이번 달 기준금리를 결정합니다. 현재 1.25%인 기준금리를 0.25%포인트 올릴 것이라는 전망과 동결할 것이라는 전망이 엇갈리고 있습니다. 최근 4%대의 가파른 물가 상승률과 미국 연방준비제도의 예상보다 빠른 긴축 가능성 때문에 인상할 것이라는 전망이 나옵니다. 다만 한국은행 총재가 공석인 데다 경기 침체나 이자 부담이 늘어날 우려에 동결 분위기도 감지됩니다 ']

In [108]:
a = ' 투자의 귀재 워런 버핏의 투자회사 버크셔 해서웨이가 최근 에너지주 중심으로 주식을 대거 매입하고 있다고 미 월스트리트저널(WSJ)이 16일(현지시간) 보도했다. 버크셔 해서웨이는 이날 미 증권거래위원회(SEC) 공시를 통해 지난주 옥시덴털 패트롤리엄 주식 90만1768주를 사들였다고 밝혔다. 지난 2월 말부터 옥시덴털 주식을 매입하기 시작해 현재는 10대 보유 종목에 들어갈 정도로 보유 지분을 늘렸다. 옥시덴털 페트롤리엄은 이날 뉴욕증권거래소에서 5.68%(3.64달러) 상승한 67.72달러에 거래를 마쳤다. 지난주 마지막 거래일인 지난 14일 8.21%의 상승률을 기록했던 주가가 이번 주 첫 거래일인 이날까지 강세를 보이고 있는 모습니다. 시장에서는 옥시덴털 페트롤리엄을 추가로 사들인 버핏의 선택이 매수세를 이끈 것으로 평가하고 있다. 이와 함께 버크셔 해서웨이는 지난 몇 달 동안 석유업체인 셰브런과 마이크로소프트(MS)가 인수를 발표한 대형 게임업체 액티비전블리자드, PC·프린터 제조사 HP, 씨티그룹과 앨리 파이낸셜의 주식을 사들였으며, 애플 지분도 계속 늘려왔다. 이에 반해 대형 은행 웰스파고의 지분은 대부분 정리한 것으로 알려졌다. 평소 버핏이 투자자들에게 강조한 "남들이 두려워할 때 욕심을 내라"고 한 말을 실행하고 있다는 분석이다. 버핏이 투자를 늘린 에너지주는 올해 스탠더드앤드푸어스(S&P)500 지수 업종 가운데 가장 좋은 주가 흐름을 보이고 있다. 올해 들어 S&P500 지수는 16% 하락했으나 옥시덴털과 셰브런 주가는 각각 134%와 47% 치솟았다. 아리엘 인베스트먼츠의 루팔 반살리는 버핏의 셰브런과 옥시덴털 투자는 원자재 가격이 당분간 고공행진할 것으로 봤기 때문일 것이라면서 인플레이션 위험 회피의 의미도 있어 보인다고 평가했다. 한편 버핏은 지난 2월 버크셔 해서웨이 주주들에게 보낸 서한에서 3월 말 기준 현금보유액이 1063억 달러(약 135조8195억원)로 지난해 말의 1467억 달러(약 187조 4385억원)에 비해 감소했다고 밝힌 바 있다. '

In [109]:
b = a.replace('니다', '니다. ')
b = b.replace('요 ', ' 요. ')
b = b.replace('. .', '.')
b = b.replace('-(앵커)', '')
b = b.replace('-(기자)', '')
b

' 투자의 귀재 워런 버핏의 투자회사 버크셔 해서웨이가 최근 에너지주 중심으로 주식을 대거 매입하고 있다고 미 월스트리트저널(WSJ)이 16일(현지시간) 보도했다. 버크셔 해서웨이는 이날 미 증권거래위원회(SEC) 공시를 통해 지난주 옥시덴털 패트롤리엄 주식 90만1768주를 사들였다고 밝혔다. 지난 2월 말부터 옥시덴털 주식을 매입하기 시작해 현재는 10대 보유 종목에 들어갈 정도로 보유 지분을 늘렸다. 옥시덴털 페트롤리엄은 이날 뉴욕증권거래소에서 5.68%(3.64달러) 상승한 67.72달러에 거래를 마쳤다. 지난주 마지막 거래일인 지난 14일 8.21%의 상승률을 기록했던 주가가 이번 주 첫 거래일인 이날까지 강세를 보이고 있는 모습니다. 시장에서는 옥시덴털 페트롤리엄을 추가로 사들인 버핏의 선택이 매수세를 이끈 것으로 평가하고 있다. 이와 함께 버크셔 해서웨이는 지난 몇 달 동안 석유업체인 셰브런과 마이크로소프트(MS)가 인수를 발표한 대형 게임업체 액티비전블리자드, PC·프린터 제조사 HP, 씨티그룹과 앨리 파이낸셜의 주식을 사들였으며, 애플 지분도 계속 늘려왔다. 이에 반해 대형 은행 웰스파고의 지분은 대부분 정리한 것으로 알려졌다. 평소 버핏이 투자자들에게 강조한 "남들이 두려워할 때 욕심을 내라"고 한 말을 실행하고 있다는 분석이다. 버핏이 투자를 늘린 에너지주는 올해 스탠더드앤드푸어스(S&P)500 지수 업종 가운데 가장 좋은 주가 흐름을 보이고 있다. 올해 들어 S&P500 지수는 16% 하락했으나 옥시덴털과 셰브런 주가는 각각 134%와 47% 치솟았다. 아리엘 인베스트먼츠의 루팔 반살리는 버핏의 셰브런과 옥시덴털 투자는 원자재 가격이 당분간 고공행진할 것으로 봤기 때문일 것이라면서 인플레이션 위험 회피의 의미도 있어 보인다고 평가했다. 한편 버핏은 지난 2월 버크셔 해서웨이 주주들에게 보낸 서한에서 3월 말 기준 현금보유액이 1063억 달러(약 135조8195억원)로 지난해 말의 1467억 달러(약 187조 4385억원)에 비해 감소했다고

In [110]:
text_list = b.split('.')
text_list

[' 투자의 귀재 워런 버핏의 투자회사 버크셔 해서웨이가 최근 에너지주 중심으로 주식을 대거 매입하고 있다고 미 월스트리트저널(WSJ)이 16일(현지시간) 보도했다',
 ' 버크셔 해서웨이는 이날 미 증권거래위원회(SEC) 공시를 통해 지난주 옥시덴털 패트롤리엄 주식 90만1768주를 사들였다고 밝혔다',
 ' 지난 2월 말부터 옥시덴털 주식을 매입하기 시작해 현재는 10대 보유 종목에 들어갈 정도로 보유 지분을 늘렸다',
 ' 옥시덴털 페트롤리엄은 이날 뉴욕증권거래소에서 5',
 '68%(3',
 '64달러) 상승한 67',
 '72달러에 거래를 마쳤다',
 ' 지난주 마지막 거래일인 지난 14일 8',
 '21%의 상승률을 기록했던 주가가 이번 주 첫 거래일인 이날까지 강세를 보이고 있는 모습니다',
 ' 시장에서는 옥시덴털 페트롤리엄을 추가로 사들인 버핏의 선택이 매수세를 이끈 것으로 평가하고 있다',
 ' 이와 함께 버크셔 해서웨이는 지난 몇 달 동안 석유업체인 셰브런과 마이크로소프트(MS)가 인수를 발표한 대형 게임업체 액티비전블리자드, PC·프린터 제조사 HP, 씨티그룹과 앨리 파이낸셜의 주식을 사들였으며, 애플 지분도 계속 늘려왔다',
 ' 이에 반해 대형 은행 웰스파고의 지분은 대부분 정리한 것으로 알려졌다',
 ' 평소 버핏이 투자자들에게 강조한 "남들이 두려워할 때 욕심을 내라"고 한 말을 실행하고 있다는 분석이다',
 ' 버핏이 투자를 늘린 에너지주는 올해 스탠더드앤드푸어스(S&P)500 지수 업종 가운데 가장 좋은 주가 흐름을 보이고 있다',
 ' 올해 들어 S&P500 지수는 16% 하락했으나 옥시덴털과 셰브런 주가는 각각 134%와 47% 치솟았다',
 ' 아리엘 인베스트먼츠의 루팔 반살리는 버핏의 셰브런과 옥시덴털 투자는 원자재 가격이 당분간 고공행진할 것으로 봤기 때문일 것이라면서 인플레이션 위험 회피의 의미도 있어 보인다고 평가했다',
 ' 한편 버핏은 지난 2월 버크셔 해서웨이 주주들에게 보낸 서한에서 3월 말 기준 현금보유액이

In [111]:
text = b
text

' 투자의 귀재 워런 버핏의 투자회사 버크셔 해서웨이가 최근 에너지주 중심으로 주식을 대거 매입하고 있다고 미 월스트리트저널(WSJ)이 16일(현지시간) 보도했다. 버크셔 해서웨이는 이날 미 증권거래위원회(SEC) 공시를 통해 지난주 옥시덴털 패트롤리엄 주식 90만1768주를 사들였다고 밝혔다. 지난 2월 말부터 옥시덴털 주식을 매입하기 시작해 현재는 10대 보유 종목에 들어갈 정도로 보유 지분을 늘렸다. 옥시덴털 페트롤리엄은 이날 뉴욕증권거래소에서 5.68%(3.64달러) 상승한 67.72달러에 거래를 마쳤다. 지난주 마지막 거래일인 지난 14일 8.21%의 상승률을 기록했던 주가가 이번 주 첫 거래일인 이날까지 강세를 보이고 있는 모습니다. 시장에서는 옥시덴털 페트롤리엄을 추가로 사들인 버핏의 선택이 매수세를 이끈 것으로 평가하고 있다. 이와 함께 버크셔 해서웨이는 지난 몇 달 동안 석유업체인 셰브런과 마이크로소프트(MS)가 인수를 발표한 대형 게임업체 액티비전블리자드, PC·프린터 제조사 HP, 씨티그룹과 앨리 파이낸셜의 주식을 사들였으며, 애플 지분도 계속 늘려왔다. 이에 반해 대형 은행 웰스파고의 지분은 대부분 정리한 것으로 알려졌다. 평소 버핏이 투자자들에게 강조한 "남들이 두려워할 때 욕심을 내라"고 한 말을 실행하고 있다는 분석이다. 버핏이 투자를 늘린 에너지주는 올해 스탠더드앤드푸어스(S&P)500 지수 업종 가운데 가장 좋은 주가 흐름을 보이고 있다. 올해 들어 S&P500 지수는 16% 하락했으나 옥시덴털과 셰브런 주가는 각각 134%와 47% 치솟았다. 아리엘 인베스트먼츠의 루팔 반살리는 버핏의 셰브런과 옥시덴털 투자는 원자재 가격이 당분간 고공행진할 것으로 봤기 때문일 것이라면서 인플레이션 위험 회피의 의미도 있어 보인다고 평가했다. 한편 버핏은 지난 2월 버크셔 해서웨이 주주들에게 보낸 서한에서 3월 말 기준 현금보유액이 1063억 달러(약 135조8195억원)로 지난해 말의 1467억 달러(약 187조 4385억원)에 비해 감소했다고

In [112]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [113]:
kkma = Kkma()

# text입력받아 kkma.sentences()를 이용해 문장단위로 나는 뒤 sentences로 리턴
def text2sentences(text):
    sentences= kkma.sentences(text)  # text일 때 문장별로 리스트 만듦
    for idx in range(0, len(sentences)): #길이에 따라 문장 합침
        if len(sentences[idx]) <= 10:
            sentences[idx-1] += (' ' + sentences[idx])
            sentences[idx] = ''
    return sentences

In [114]:
#기사본문 가져오기
sentences = text2sentences(text)
sentences

['투자의 귀재 워 런 버핏의 투자회사 버크셔 해서웨이가 최근 에너지 주 중심으로 주식을 대거 매입하고 있다고',
 '미 월 스트리트 저널 (WSJ) 이 16일( 현지시간) 보도했다.',
 '버크셔 해서웨이는 이날 미 증권거래위원회 (SEC) 공시를 통해 지난주 옥시 덴 털 패트롤 리 엄 주식 90만 1768 주를 사들였다고 밝혔다.',
 '지난 2월 말부터 옥시 덴 털 주식을 매입하기 시작해 현재는 10대 보유 종목에 들어갈 정도로 보유 지분을 늘렸다.',
 '옥시 덴 털 페트롤 리 엄은 이날 뉴욕 증권거래소에서 5.68% (3.64 달러) 상승한 67.72 달러에 거래를 마쳤다.',
 '지난주 마지막 거래 일인 지난 14일 8.21% 의 상승률을 기록했던 주가가 이번 주 첫 거래 일인 이날까지 강세를 보이고 있는 모습니다.',
 '시장에서는 옥시 덴 털 페트롤 리 엄을 추가로 사들인 버핏의 선택이 매수세를 이끈 것으로 평가하고 있다.',
 '이와 함께 버크셔 해서웨이는 지난 몇 달 동안 석유업체인 셰브런과 마이크로 소프트 (MS) 가 인수를 발표한 대형 게임업체 액 티 비전 블 리 자드, PC· 프린터 제조사 HP, 씨티그룹과 앨리 파이낸셜의 주식을 사들였으며, 애플 지분도 계속 늘려 왔다.',
 '이에 반해 대형 은행 웰 스파고의 지분은 대부분 정리한 것으로 알려 졌다.',
 '평소 버핏이 투자자들에게 강조한 " 남들이 두려워할 때 욕심을 내라" 고 한 말을 실행하고 있다는 분석이다.',
 '버핏이 투자를 늘린 에너지 주는 올해 스탠더드 앤드 푸어 스 (S &P )500 지수 업종 가운데 가장 좋은 주가 흐름을 보이고 있다.',
 '올해 들어 S&P500 지수는 16% 하락했으나 옥시 덴 털과 셰브 런 주가는 각각 134% 와 47% 치솟았다.',
 '아리엘 인베스트먼츠의 루 팔 반 살리는 버핏의 셰브런과 옥시 덴 털 투자는 원자재 가격이 당분간 고공 행진할 것으로 봤기 때문일 것이라면서 인플레이션 위험 회피의 의미도 있어 보인다고 평가했다.',
 '한편

In [115]:
twitter = Twitter()
from konlpy.tag import Okt
okt = Okt()

#불용어제거
stopwords = ['( )','안녕하세요', '아', '으', '네' ,'코']
def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence != '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                  if noun not in stopwords and len(noun) > 1]))
    return nouns

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [116]:
nouns = get_nouns(sentences)

In [117]:
nouns = get_nouns(sentences)
print(sentences[0])
print(nouns[0])

투자의 귀재 워 런 버핏의 투자회사 버크셔 해서웨이가 최근 에너지 주 중심으로 주식을 대거 매입하고 있다고
투자 버핏 투자 회사 버크셔 해서웨이 최근 에너지 중심 주식 대거 매입


# TF-IDF 모델 생성 및 그래프 생성

In [118]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [119]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

sent_graph = build_sent_graph(nouns)
sent_graph

array([[1.        , 0.        , 0.17670982, 0.13112213, 0.        ,
        0.        , 0.04800316, 0.10085308, 0.        , 0.04339061,
        0.22448902, 0.        , 0.123033  , 0.07313174],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.17670982, 0.        , 1.        , 0.09708922, 0.20510547,
        0.17702149, 0.0530762 , 0.11151135, 0.        , 0.        ,
        0.        , 0.05183348, 0.0304192 , 0.04742422],
       [0.13112213, 0.        , 0.09708922, 1.        , 0.03909883,
        0.04164404, 0.04986946, 0.06499707, 0.        , 0.        ,
        0.        , 0.04870183, 0.02858135, 0.18234164],
       [0.        , 0.        , 0.20510547, 0.03909883, 1.        ,
        0.1056644 , 0.16504152, 0.        , 0.        , 0.        ,
        0.        , 0.05048036, 0.0296251 , 0.28562862],
       [0.        , 0.        , 0.1

In [120]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

In [121]:
words_graph, idx2word = build_words_graph(nouns)

In [122]:
words_graph

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 0., 1.]])

In [123]:
idx2word

{92: '투자',
 32: '버핏',
 107: '회사',
 31: '버크셔',
 102: '해서웨이',
 89: '최근',
 61: '에너지',
 83: '중심',
 81: '주식',
 20: '대거',
 27: '매입',
 47: '스트리트',
 106: '현지',
 50: '시간',
 34: '보도',
 84: '증권',
 7: '거래',
 67: '위원회',
 13: '공시',
 91: '통해',
 86: '지난주',
 62: '옥시',
 94: '트롤',
 85: '지난',
 51: '시작',
 105: '현재',
 35: '보유',
 79: '종목',
 76: '정도',
 96: '페트롤',
 17: '뉴욕',
 8: '거래소',
 18: '달러',
 39: '상승',
 26: '마지막',
 74: '일인',
 49: '승률',
 15: '기록',
 80: '주가',
 71: '이번',
 5: '강세',
 36: '보이',
 29: '모습',
 52: '시장',
 90: '추가',
 42: '선택',
 97: '평가',
 23: '동안',
 41: '석유',
 60: '업체',
 44: '셰브런',
 25: '마이크로',
 45: '소프트',
 72: '인수',
 30: '발표',
 22: '대형',
 9: '게임',
 75: '자드',
 99: '프린터',
 78: '조사',
 54: '씨티',
 14: '그룹',
 58: '앨리',
 95: '파이낸셜',
 56: '애플',
 37: '분도',
 10: '계속',
 69: '은행',
 48: '스파',
 12: '고의',
 21: '대부분',
 77: '정리',
 98: '평소',
 93: '투자자',
 6: '강조',
 64: '욕심',
 53: '실행',
 38: '분석',
 63: '올해',
 46: '스탠더드',
 57: '앤드',
 88: '지수',
 59: '업종',
 1: '가운데',
 2: '가장',
 109: '흐름',
 100: '하락',
 43: '셰브',
 3: '각각',


In [124]:
for key, value in idx2word.items():
    if value == '때문':
        print(key)

24


In [125]:
def get_ranks(graph, d=0.85):
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0
        link_sum = np.sum(A[:,id])
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1
        
    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B)
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [126]:
sent_rank_idx = get_ranks(sent_graph)
sent_rank_idx

{0: 1.4047005213097443,
 1: 0.15000000000000002,
 2: 1.4242032978821177,
 3: 1.0592374044138737,
 4: 1.2983393906222889,
 5: 0.875042724852007,
 6: 1.0700516913707339,
 7: 0.8791563584321606,
 8: 0.2529105787178386,
 9: 0.43478830072870045,
 10: 1.2199066685057607,
 11: 0.9266436284730908,
 12: 0.9273553978890268,
 13: 1.2276640368026561}

In [127]:
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)
sorted_sent_rank_idx

[2, 0, 4, 13, 10, 6, 3, 12, 11, 7, 5, 9, 8, 1]

In [128]:
word_rank_idx = get_ranks(words_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)

# 요약하기 실행

In [129]:
llist=[]
summary = []
summarize=[]
index = []
for idx in sorted_sent_rank_idx[:1]:
    index.append(idx)
        
index.append(idx)
    
for idx in index:
    summary.append(sentences[idx])
        
for text in summary[:]:
    llist.append(text)
    
m = list(set(llist))

for i in m:
    i = i.replace('-( )', '')
    summarize.append(i)

In [130]:
llist=[]
summary = []
summarize=[]
index = []
for idx in sorted_sent_rank_idx[:3]:
    index.append(idx)
        
index.append(idx)
    
for idx in index:
    summary.append(sentences[idx])
        
for text in summary[:]:
    llist.append(text)
    
m = list(set(llist))

for i in m:
    i = i.replace('-( )', '')
    summarize.append(i)

In [131]:
# 1문장 요약
summarize

['투자의 귀재 워 런 버핏의 투자회사 버크셔 해서웨이가 최근 에너지 주 중심으로 주식을 대거 매입하고 있다고',
 '옥시 덴 털 페트롤 리 엄은 이날 뉴욕 증권거래소에서 5.68% (3.64 달러) 상승한 67.72 달러에 거래를 마쳤다.',
 '버크셔 해서웨이는 이날 미 증권거래위원회 (SEC) 공시를 통해 지난주 옥시 덴 털 패트롤 리 엄 주식 90만 1768 주를 사들였다고 밝혔다.']

In [82]:
# 3문장 요약
summarize

['현재 유럽에서는 암호 화폐가 최소한의 환경적 지속 가능성 기준을 따라야 한다며 이런 작업 증명 방식을 채택한 암호 화폐를 사실상 금지하는 방안이 논의 중이다.',
 '암호 화폐 거래소 ‘FTX ’를 창업한 샘 뱅크 맨- 프라이드 최고 경영자 (CEO) 가 비트 코 인이 지급 결제 네트워크로서 미래가 없다고 말했다고',
 '그는 작업 증명 방식보다는 지분 증명 (PoS) 방식이 더 싸고 전력을 덜 소비하기 때문에 암호 화폐가 지급 결제 네트워크로 진화하는 데 더 적합한 방식이라고 주장했다.']

In [57]:
summary = ' '.join(summarize)
summary

'정답은 나무가 있는 숲입니다. 그렇다면 마스크에 위축된 아이들의 뇌 잘 발달하도록 자극할 방법 없을까요? 나무가 있는 숲과 강 그리고 호수와 잔디, 집 근처 어떤 자연 환경이 있을 때 아이들 뇌가 잘 발달할까요? 호수일까요?'

In [51]:
summarize

['뇌 두께가 더 두꺼울수록 뇌가 더 발달했다고',
 '정답은 나무가 있는 숲입니다.',
 '나무가 있는 숲과 강 그리고 호수와 잔디, 집 근처 어떤 자연 환경이 있을 때 아이들 뇌가 잘 발달할까요? 호수일까요?',
 '멀리 보이는 숲은 어린이에게 지혜를 주고 가까이 닿는 나무는 어린이의 다친 마음을 치유해 준다는 겁니다.',
 '그렇다면 마스크에 위축된 아이들의 뇌 잘 발달하도록 자극할 방법 없을까요?']

In [52]:
summary = ' '.join(summarize)
summary

'뇌 두께가 더 두꺼울수록 뇌가 더 발달했다고 정답은 나무가 있는 숲입니다. 나무가 있는 숲과 강 그리고 호수와 잔디, 집 근처 어떤 자연 환경이 있을 때 아이들 뇌가 잘 발달할까요? 호수일까요? 멀리 보이는 숲은 어린이에게 지혜를 주고 가까이 닿는 나무는 어린이의 다친 마음을 치유해 준다는 겁니다. 그렇다면 마스크에 위축된 아이들의 뇌 잘 발달하도록 자극할 방법 없을까요?'

In [53]:
stop_words = ['때문', '지금', '이', '을', '의', '는', '가', '들', ',', '도',
              '은', '그', '.', '를', '너무', '고', '것' ,'로', '하는','보고','에','때','에',
              '저','하고','에서','정말','못','다','까지','또','개','수','으로','만','할',
              '과','인','님','-','임','7','봐도','하는거','그냥','해도','바로','순간',
              '잘','한','겁나','나','같이','함께','라고','쓴','아니라','아닌','옹','살','진짜','하는게','같아요','제목','적', '가야','지금',
             '관련']
    
keywords = []
index = []
for idx in sorted_word_rank_idx[:15]:
    index.append(idx)
        
for idx in index:
    keywords.append(idx2word[idx])
        
keywords = [each_word for each_word in keywords
            if each_word not in stop_words]
        
keyword = keywords[:5]

In [54]:
keyword

['아이', '분석', '발달', '연구', '동안']

def summarize(sent_num=5):
    summary = []
    index = []
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)
        
    index.append(idx)
    
    for idx in index:
        summary.append(sentences[idx])
        
    for text in summary[:]:
        print(text)